1 - get the nodes state observations : their informations through the rounds + the accuracy of their models

In [4]:
import json
# Load the JSON data from the file scratch/subdir/results/score_alpha02_nodes_150/blockchain.json
# with open('../results/score_alpha02_nodes_150/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_alpha05_nodes_150/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_alpha08_nodes150/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_dropout10/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_dropout30/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_dropout60/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_malicious10/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_malicious30/blockchain.json', 'r') as f:
#     data = json.load(f)
with open('../results/score_01_06/blockchain.json', 'r') as f:
    data = json.load(f)

# Create a list to store node information for each block
node_info_by_block = {}
transitions = []
nodes_by_block=[]
# transition = {"obs" : , "obs_" , "action" : , "reward" : ,"done" : }
# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']

    for transaction in json_obj['Transactions']:
        content = transaction.get('Content', {})
        message_type = content.get('message_type')
        node_id = content.get('nodeId')

        if message_type == 1 and node_id is not None:
            availability = int(content.get('availability'))
            honesty = content.get('honesty')
            dataset_size = content.get('datasetSize')
            freq = content.get('freq')
            trans_rate = content.get('transRate')
            task = content.get('task')
            dropout = content.get('dropout', 0.0)

            # Find accuracy for the corresponding node in message_type 2 transactions
            accuracy = 0.0
            for json_obj2 in data:
                if json_obj2['BlockId'] == block_id:
                    for transaction2 in json_obj2['Transactions']:
                        if transaction2['Content'].get('message_type') == 2 and transaction2['Content'].get('nodeId') == node_id and transaction2['Content'].get('type') == 0:
                            accuracy = transaction2['Content'].get('accuracy', 0.0)
                            break
                    if accuracy != 0.0:
                        break

            node_info = [node_id, availability, honesty, dataset_size, freq, trans_rate, dropout, accuracy]

            if block_id not in node_info_by_block:
                node_info_by_block[block_id] = []

            node_info_by_block[block_id].append(node_info)

# Sort the node lists within each block by nodeId
for nodes in node_info_by_block.values():
    nodes.sort(key=lambda x: x[0])  # Sort based on nodeId

# Print or use node_info_by_block as needed
for block_id, nodes in node_info_by_block.items():
    print(f"Block {block_id}:")
    nodes_by_block.append(nodes)
    for node_info in nodes:
        print(node_info)
   


Block 1/0:
[0, 1, 0.0, 370, 190.0, 920.0, 0.0, 41.89]
[1, 1, 0.0, 190, 210.0, 190.0, 1.0, 0.0]
[2, 0, 0.0, 330, 110.0, 660.0, 0.0, 0.0]
[3, 1, 0.0, 60, 150.0, 440.0, 0.0, 25.0]
[4, 1, 0.0, 150, 160.0, 520.0, 0.0, 26.67]
[5, 1, 0.0, 400, 80.0, 160.0, 1.0, 0.0]
[6, 1, 0.0, 180, 290.0, 940.0, 0.0, 0.0]
[7, 1, 0.0, 380, 60.0, 260.0, 1.0, 0.0]
[8, 1, 0.0, 270, 110.0, 770.0, 0.0, 62.96]
[9, 1, 0.0, 130, 190.0, 670.0, 0.0, 0.0]
[10, 1, 0.0, 410, 210.0, 150.0, 0.0, 0.0]
[11, 1, 0.0, 330, 130.0, 570.0, 0.0, 45.45]
[12, 1, 0.0, 160, 240.0, 220.0, 0.0, 0.0]
[13, 1, 0.0, 140, 110.0, 160.0, 0.0, 0.0]
[14, 1, 0.0, 300, 80.0, 210.0, 0.0, 0.0]
[15, 1, 0.0, 140, 150.0, 700.0, 0.0, 0.0]
[16, 1, 0.0, 50, 60.0, 960.0, 0.0, 0.0]
[17, 1, 0.0, 210, 170.0, 650.0, 0.0, 54.76]
[18, 1, 0.0, 500, 270.0, 180.0, 0.0, 0.0]
[19, 1, 0.0, 170, 50.0, 380.0, 0.0, 0.0]
[20, 0, 0.0, 60, 150.0, 710.0, 0.0, 0.0]
[21, 1, 0.0, 140, 230.0, 900.0, 0.0, 0.0]
[22, 1, 0.0, 460, 270.0, 860.0, 0.0, 0.0]
[23, 1, 0.0, 470, 220.0, 550.0

2 - get the global accuracy for each round

In [5]:
# Create a list to store the global accuracy values
global_accuracy = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 2 and type equal to 2
    accuracy_transaction = None
    for transaction in json_obj['Transactions']:
        content = transaction['Content']
        if content.get('message_type') == 2 and content.get('type') == 2:
            accuracy_transaction = transaction
            break
    
    if accuracy_transaction is not None:
        accuracy = accuracy_transaction['Content'].get('accuracy', 0.0)
        global_accuracy.append(accuracy)

# Print or use global_accuracy as needed
print("Global Accuracy Values:", global_accuracy)

Global Accuracy Values: [43.89, 64.0, 64.0, 73.64, 75.45, 75.45, 78.18, 80.0, 80.0, 81.15, 83.64, 81.82, 81.82, 90.0, 90.0, 85.38, 88.33, 86.15, 89.23, 90.0, 88.08, 85.38, 87.69, 88.46, 88.85, 88.08, 85.77, 89.23, 86.15]


In [6]:
# Create a list to store the concatenated lists for each block
selected_nodes = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes.append(aggregators + trainers)

# Print or use selected_nodes as needed
for block_id, selected_node_list in zip([json_obj['BlockId'] for json_obj in data], selected_nodes):
    print(f"Block {block_id}: {selected_node_list}")

Block 1/0: [6, 21, 16, 35, 38, 40, 9, 46, 15, 22, 26, 27, 0, 17, 42, 30, 4, 3, 33, 39, 45, 8, 29, 44, 34, 23, 11, 47, 24, 37]
Block 1/1: [16, 35, 40, 9, 15, 22, 21, 26, 20, 27, 0, 17, 28, 3, 25, 33, 29, 34, 11, 38, 2, 24, 37, 43, 19, 12, 48, 1, 32, 18]
Block 1/2: [16, 35, 9, 15, 21, 26, 20, 27, 40, 0, 28, 3, 6, 29, 4, 11, 44, 37, 19, 12, 48, 41, 13, 49, 7, 10, 36, 46, 42, 39]
Block 1/3: [35, 16, 15, 26, 20, 27, 40, 9, 28, 3, 6, 29, 11, 44, 22, 37, 41, 13, 14, 0, 36, 45, 5, 23, 47, 43, 1, 7, 42, 39]
Block 1/4: [35, 20, 16, 40, 21, 26, 9, 6, 29, 3, 11, 44, 22, 27, 48, 13, 0, 31, 46, 45, 5, 23, 47, 43, 1, 7, 42, 39, 8, 33]
Block 1/5: [35, 20, 16, 40, 15, 21, 26, 9, 6, 11, 28, 29, 3, 44, 22, 27, 37, 19, 12, 48, 13, 0, 31, 36, 46, 45, 5, 47, 43, 1]
Block 1/6: [35, 20, 40, 6, 28, 29, 44, 11, 37, 12, 48, 13, 0, 31, 36, 23, 7, 46, 42, 39, 45, 5, 47, 43, 17, 14, 24, 4, 18, 38]
Block 1/7: [35, 40, 16, 20, 15, 21, 26, 9, 6, 29, 44, 11, 22, 27, 37, 48, 19, 12, 0, 31, 23, 7, 46, 42, 39, 45, 5, 47, 

In [7]:
round_reward = []
transitions = []

for block_id in range(len(node_info_by_block) - 1):  # Fixed the range
    # print(len(node_info_by_block) - 1)
    # print(f"Block {block_id}:")
    nodes_rewards = []
    
    for node_index in selected_nodes[block_id]:
        # print("Node index: ", node_index)
        node_data = nodes_by_block[block_id][node_index]
        reward = node_data[-1]
        # print("Original reward: ", reward)
        
        if reward != 0.0:
            reward *= node_data[2]
        else :
            reward = node_data[2]
        # print("Modified reward: ", reward)
        
        nodes_rewards.append(reward)
    
    # print("Nodes rewards:", nodes_rewards)
    cumulated_reward = sum(nodes_rewards)
    round_reward.append(cumulated_reward)
    
    transition = {
        "obs": nodes_by_block[block_id],
        "obs_": nodes_by_block[block_id + 1],
        "action": selected_nodes[block_id],
        "reward": cumulated_reward,
        "done": block_id == len(node_info_by_block) - 1  # Corrected the condition
    }
    transitions.append(transition)

for transition in transitions : 
    print(transition)


{'obs': [[0, 1, 0.0, 370, 190.0, 920.0, 0.0, 41.89], [1, 1, 0.0, 190, 210.0, 190.0, 1.0, 0.0], [2, 0, 0.0, 330, 110.0, 660.0, 0.0, 0.0], [3, 1, 0.0, 60, 150.0, 440.0, 0.0, 25.0], [4, 1, 0.0, 150, 160.0, 520.0, 0.0, 26.67], [5, 1, 0.0, 400, 80.0, 160.0, 1.0, 0.0], [6, 1, 0.0, 180, 290.0, 940.0, 0.0, 0.0], [7, 1, 0.0, 380, 60.0, 260.0, 1.0, 0.0], [8, 1, 0.0, 270, 110.0, 770.0, 0.0, 62.96], [9, 1, 0.0, 130, 190.0, 670.0, 0.0, 0.0], [10, 1, 0.0, 410, 210.0, 150.0, 0.0, 0.0], [11, 1, 0.0, 330, 130.0, 570.0, 0.0, 45.45], [12, 1, 0.0, 160, 240.0, 220.0, 0.0, 0.0], [13, 1, 0.0, 140, 110.0, 160.0, 0.0, 0.0], [14, 1, 0.0, 300, 80.0, 210.0, 0.0, 0.0], [15, 1, 0.0, 140, 150.0, 700.0, 0.0, 0.0], [16, 1, 0.0, 50, 60.0, 960.0, 0.0, 0.0], [17, 1, 0.0, 210, 170.0, 650.0, 0.0, 54.76], [18, 1, 0.0, 500, 270.0, 180.0, 0.0, 0.0], [19, 1, 0.0, 170, 50.0, 380.0, 0.0, 0.0], [20, 0, 0.0, 60, 150.0, 710.0, 0.0, 0.0], [21, 1, 0.0, 140, 230.0, 900.0, 0.0, 0.0], [22, 1, 0.0, 460, 270.0, 860.0, 0.0, 0.0], [23, 1, 0

In [15]:
import sys
sys.path.insert(0, '../')  # Add the directory containing "DRL" to the path

from agent import node_selection_agent as ag
from environment import node_selection_env as nds

total_nodes = 50
select_trainer = 20
select_Agg = 10
num_features = 8
target_accuracy = 99
max_rounds= 30

#create the nodes selection environment accoding to the observation
envNodeSelect = nds.FLNodeSelectionEnv(total_nodes,select_Agg,select_trainer, num_features=num_features, target=target_accuracy, max_rounds=max_rounds)
obs_shape = envNodeSelect.observation_space.sample()["current_state"].shape
print(obs_shape)
agent = ag.Agent(input_shape=obs_shape ,n_actions=obs_shape[0], env=envNodeSelect,max_actions=select_trainer+select_Agg) 
agent.load_models()

# train the agent with the transitions store the trasitions then make the agent learn
for transition in transitions : 
    agent.remember(transition["obs"],transition["action"], transition["reward"], transition["obs_"], transition["done"])
    agent.learn()
    agent.load_models()

agent.save_models()


(50, 8)
.... loading models ....
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
.... finished loading models ....
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... loading models ....
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
.... finished loading models ....
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... loading models ....
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
.... finished loading models ....
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving chec

In [9]:
from collections import defaultdict


# Create a dictionary to store the concatenated lists for each block
selected_nodes = defaultdict(list)

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes[block_id] = aggregators + trainers

# Count how many times each node was selected
node_selection_counts = defaultdict(int)
for selected_node_list in selected_nodes.values():
    for node_id in selected_node_list:
        node_selection_counts[node_id] += 1

# Print or use node_selection_counts as needed
for node_id, count in node_selection_counts.items():
    print(f"Node {node_id} was selected {count} times.")


Node 6 was selected 27 times.
Node 21 was selected 25 times.
Node 16 was selected 28 times.
Node 35 was selected 28 times.
Node 38 was selected 5 times.
Node 40 was selected 29 times.
Node 9 was selected 27 times.
Node 46 was selected 22 times.
Node 15 was selected 25 times.
Node 22 was selected 27 times.
Node 26 was selected 27 times.
Node 27 was selected 28 times.
Node 0 was selected 27 times.
Node 17 was selected 6 times.
Node 42 was selected 20 times.
Node 30 was selected 4 times.
Node 4 was selected 6 times.
Node 3 was selected 11 times.
Node 33 was selected 8 times.
Node 39 was selected 22 times.
Node 45 was selected 21 times.
Node 8 was selected 4 times.
Node 29 was selected 26 times.
Node 44 was selected 26 times.
Node 34 was selected 6 times.
Node 23 was selected 22 times.
Node 11 was selected 16 times.
Node 47 was selected 21 times.
Node 24 was selected 6 times.
Node 37 was selected 24 times.
Node 20 was selected 27 times.
Node 28 was selected 25 times.
Node 25 was selected 4